<a href="https://colab.research.google.com/github/imjunyoung404/ai-class-3-2/blob/main/3%EC%A3%BC%EC%B0%A8/%EC%A0%84%EB%B3%B5%20%EB%B6%84%EB%A5%98%EB%A5%BC%20CNN%EC%9C%BC%EB%A1%9C%20%EA%B5%AC%ED%98%84_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd

# CSV 파일 읽어오기
df = pd.read_csv("/content/abalone (1).csv")

# 데이터 확인
print(df.head())

   id Sex  Length  Diameter  Height  Whole_weight  Shucked_weight  \
0   0   M   0.455     0.365   0.095        0.5140          0.2245   
1   1   M   0.350     0.265   0.090        0.2255          0.0995   
2   2   F   0.530     0.420   0.135        0.6770          0.2565   
3   3   M   0.440     0.365   0.125        0.5160          0.2155   
4   4   I   0.330     0.255   0.080        0.2050          0.0895   

   Viscera_weight  Shell_weight  Rings  
0          0.1010         0.150     15  
1          0.0485         0.070      7  
2          0.1415         0.210      9  
3          0.1140         0.155     10  
4          0.0395         0.055      7  


In [79]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# 'Sex' 열을 숫자로 변환 (예: M -> 1, F -> 2, I -> 0)
label_encoder = LabelEncoder()
df['Sex'] = label_encoder.fit_transform(df['Sex'])

# 피처와 타겟 설정
X = df.drop('Rings', axis=1).values  # 입력 데이터
y = df['Rings'].values  # 타겟 값

# 데이터 스케일링
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 데이터 분할 (80% 훈련, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# CNN 모델 정의
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # 'Rings'는 회귀 문제이므로 'linear' 출력층 사용
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 학습
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

# 모델 평가
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae:.2f}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 63.3596 - mae: 6.7188 - val_loss: 7.4931 - val_mae: 2.0164
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9.5042 - mae: 2.1271 - val_loss: 5.6544 - val_mae: 1.6850
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.8514 - mae: 1.9577 - val_loss: 5.2733 - val_mae: 1.6277
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7.6107 - mae: 1.9423 - val_loss: 5.0232 - val_mae: 1.6014
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 6.7753 - mae: 1.8530 - val_loss: 5.0968 - val_mae: 1.6008
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 6.7403 - mae: 1.8557 - val_loss: 4.8997 - val_mae: 1.5561
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 6.0990 - mae: 1.8091 - val_loss: 4.7901 - val_mae: 1.5561
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 6.4844 - mae: 1.8444 - val_loss: 4.6487 - val_mae: 1.5401
Epoch 9/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - l

In [81]:
# 학습된 모델 저장
model.save('abalone_cnn_model.h5')

In [84]:
# 저장된 모델 불러오기
pretrained_model = tf.keras.models.load_model('abalone_cnn_model.h5', custom_objects={'mse': tf.keras.losses.MeanSquaredError}) # explicitly define the 'mse'

# 사전학습 모델의 마지막 몇 개 층을 동결
for layer in pretrained_model.layers[:-1]:  # 마지막 층만 동결하지 않음
    layer.trainable = False

# 새로운 층 추가
new_model = Sequential(pretrained_model.layers)
new_model.add(Dense(128, activation='relu'))
new_model.add(Dense(1, activation='linear'))  # 회귀 문제이므로 'linear'

# 새로운 모델 컴파일
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='mse', metrics=['mae'])

In [85]:
# 미세조정 학습
history_fine = new_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# 모델 평가
test_loss, test_mae = new_model.evaluate(X_test, y_test)
print(f'Fine-tuned Test MAE: {test_mae:.2f}')

Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 168.6919 - mae: 12.4511 - val_loss: 141.6327 - val_mae: 11.3441
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 133.2630 - mae: 11.0215 - val_loss: 112.1434 - val_mae: 10.0460
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 105.3265 - mae: 9.7543 - val_loss: 87.6042 - val_mae: 8.8219
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 81.2358 - mae: 8.4953 - val_loss: 65.8854 - val_mae: 7.5766
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 61.3425 - mae: 7.3084 - val_loss: 46.8709 - val_mae: 6.2891
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 41.8014 - mae: 5.9509 - val_loss: 31.0960 - val_mae: 4.9774
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 28.4925 - mae: 4.7251 - val_loss: 19.4489 - val_mae: 3.7302
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 17.0680 - mae: 3.4428 - val_loss: 11.8798 - val_mae: 2.6627
Epoch 9/20
105/105 ━━━━━━━━━━━━